In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model
import cv2

In [2]:
train = r"C:\Users\LIYANA\Downloads\emotion-detection-fer\train"
test = r"C:\Users\LIYANA\Downloads\emotion-detection-fer\test"

In [3]:
def load_images(folder):
    img = []
    label = []
    for sub in os.listdir(folder):
        subpath = os.path.join(folder, sub)
        if os.path.isdir(subpath):
            lab = sub
            for name in os.listdir(subpath):
                imgpath = os.path.join(subpath, name)
                im = cv2.imread(imgpath)

                if im is not None:
                    img.append(im)
                    label.append(lab)

    return img, label

In [4]:
train_img, train_lab = load_images(train)
train_img = np.array(train_img) / 255.0
test_img, test_lab = load_images(test)
test_img = np.array(test_img) / 255.0

In [5]:
label_encoder = LabelEncoder()
train_lab = label_encoder.fit_transform(train_lab)
train_lab = to_categorical(train_lab, num_classes=7)
test_lab = label_encoder.transform(test_lab)
test_lab = to_categorical(test_lab, num_classes=7)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(train_img, train_lab, test_size=0.2, random_state=42)

In [7]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')

In [11]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), callbacks=[checkpoint, early_stopping])

Epoch 1/50
359/359 [==============================] - 104s 288ms/step - loss: 1.5166 - accuracy: 0.4282 - val_loss: 1.4562 - val_accuracy: 0.4404
Epoch 2/50
359/359 [==============================] - 96s 269ms/step - loss: 1.3947 - accuracy: 0.4704 - val_loss: 1.3489 - val_accuracy: 0.4850
Epoch 3/50
359/359 [==============================] - 97s 270ms/step - loss: 1.3285 - accuracy: 0.4936 - val_loss: 1.3011 - val_accuracy: 0.5080
Epoch 4/50
359/359 [==============================] - 95s 264ms/step - loss: 1.2794 - accuracy: 0.5122 - val_loss: 1.4513 - val_accuracy: 0.4596
Epoch 5/50
359/359 [==============================] - 96s 266ms/step - loss: 1.2412 - accuracy: 0.5277 - val_loss: 1.2366 - val_accuracy: 0.5387
Epoch 6/50
359/359 [==============================] - 96s 268ms/step - loss: 1.2010 - accuracy: 0.5419 - val_loss: 1.2510 - val_accuracy: 0.5347
Epoch 7/50
359/359 [==============================] - 108s 301ms/step - loss: 1.1709 - accuracy: 0.5565 - val_loss: 1.1889 - val_

In [12]:
best_model = load_model('best_model.h5')

In [14]:
import shutil

source = 'best_model.h5'
destination = r"C:\Users\LIYANA\OneDrive\Documents\Projects\best_model_copy.h5"  # Different filename

shutil.copyfile(source, destination)
print(f'Saved model to {destination}')

Saved model to C:\Users\LIYANA\OneDrive\Documents\Projects\best_model_copy.h5


In [ ]:
val_loss, val_accuracy = best_model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

180/180 [==============================] - 7s 33ms/step - loss: 1.1112 - accuracy: 0.6181
Validation Loss: 1.1112223863601685
Validation Accuracy: 0.6180773377418518
